## 4.3 Tensor board and Model Save

### Tensor Board
* 그래프 구조 및 변수  저장
    * 네임 스코프 지정
        * 깔끔한 분류
        * `with tf.name_scope('optimizer'):`
    * 이름 표시
        * 변수 및 오프레이터에 name 지정
        * `W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.), name='W1')`
    * 로그 파일 저장
        * `writer = tf.summary.FileWriter('./logs/dnn', tf.get_default_graph())`
* 텐서 보드 실행
    * `tensorboard --logdir=./logs`
* 웹 브라우저로 접속
    * http://localhost:6006

In [1]:
import tensorflow as tf
import numpy as np

data = np.loadtxt('./data/data.csv', delimiter=',',
                  unpack=True, dtype='float32')

x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])
print(x_data, x_data.shape, x_data.dtype)

#########
# 신경망 모델 구성
######
global_step = tf.Variable(0, trainable=False, name='global_step')

X = tf.placeholder(tf.float32, name="phX")
Y = tf.placeholder(tf.float32, name="phY")

with tf.name_scope('layer1'):
    W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.), name='W1')
    L1 = tf.nn.relu(tf.matmul(X, W1))

    tf.summary.histogram("X", X)
    tf.summary.histogram("Weights", W1)

with tf.name_scope('layer2'):
    W2 = tf.Variable(tf.random_uniform([10, 20], -1., 1.), name='W2')
    L2 = tf.nn.relu(tf.matmul(L1, W2))

    tf.summary.histogram("Weights", W2)

with tf.name_scope('output'):
    W3 = tf.Variable(tf.random_uniform([20, 3], -1., 1.), name='W3')
    model = tf.matmul(L2, W3)

    tf.summary.histogram("Weights", W3)
    tf.summary.histogram("Model", model)

with tf.name_scope('optimizer'):
    cost = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))

    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    train_op = optimizer.minimize(cost, global_step=global_step)

    tf.summary.scalar('cost', cost)

#########
# 신경망 모델 학습
######
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())

ckpt = tf.train.get_checkpoint_state('./model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    print('exist')
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    print("not exit")
    sess.run(tf.global_variables_initializer())

merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('./logs', sess.graph)

for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})

    print('Step: %d, ' % sess.run(global_step),
          'Cost: %.3f' % sess.run(cost, feed_dict={X: x_data, Y: y_data}))

    summary = sess.run(merged, feed_dict={X: x_data, Y: y_data})
    writer.add_summary(summary, global_step=sess.run(global_step))

saver.save(sess, './model/dnn.ckpt', global_step=global_step)

#########
# 결과 확인
######
prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

[[0. 0.]
 [1. 0.]
 [1. 1.]
 [0. 0.]
 [0. 0.]
 [0. 1.]] (6, 2) float32
Instructions for updating:
Colocations handled automatically by placer.
not exit
Step: 1,  Cost: 1.143
Step: 2,  Cost: 1.044
Step: 3,  Cost: 0.961
Step: 4,  Cost: 0.895
Step: 5,  Cost: 0.842
Step: 6,  Cost: 0.800
Step: 7,  Cost: 0.766
Step: 8,  Cost: 0.735
Step: 9,  Cost: 0.710
Step: 10,  Cost: 0.688
Step: 11,  Cost: 0.669
Step: 12,  Cost: 0.652
Step: 13,  Cost: 0.639
Step: 14,  Cost: 0.627
Step: 15,  Cost: 0.616
Step: 16,  Cost: 0.607
Step: 17,  Cost: 0.600
Step: 18,  Cost: 0.593
Step: 19,  Cost: 0.588
Step: 20,  Cost: 0.583
Step: 21,  Cost: 0.579
Step: 22,  Cost: 0.575
Step: 23,  Cost: 0.572
Step: 24,  Cost: 0.570
Step: 25,  Cost: 0.567
Step: 26,  Cost: 0.565
Step: 27,  Cost: 0.564
Step: 28,  Cost: 0.562
Step: 29,  Cost: 0.561
Step: 30,  Cost: 0.560
Step: 31,  Cost: 0.559
Step: 32,  Cost: 0.558
Step: 33,  Cost: 0.557
Step: 34,  Cost: 0.556
Step: 35,  Cost: 0.556
Step: 36,  Cost: 0.555
Step: 37,  Cost: 0.555
Step: 3

### 모델 저장
* `saver = tf.train.Saver(tf.global_variables())`
* `saver.save(sess, './model/dnn_final.ckpt')`

In [2]:

saver.save(sess, './model/dnn_final.ckpt')

'./model/dnn_final.ckpt'

### Tensorflow 모델 읽기


In [3]:
import tensorflow as tf
import numpy as np

# [털, 날개]
x_data = np.array(
    [[0, 0], [1, 0], [1, 1], [0, 0], [0, 0], [0, 1]])

# [기타, 포유류, 조류]
y_data = np.array([
    [1, 0, 0],  # 기타
    [0, 1, 0],  # 포유류
    [0, 0, 1],  # 조류
    [1, 0, 0],
    [1, 0, 0],
    [0, 0, 1]
])

#########
# 신경망 모델 구성
######
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

# 첫번째 가중치의 차원은 [특성, 히든 레이어의 뉴런갯수] -> [2, 10] 으로 정합니다.
W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.))
# 두번째 가중치의 차원을 [첫번째 히든 레이어의 뉴런 갯수, 분류 갯수] -> [10, 3] 으로 정합니다.
W2 = tf.Variable(tf.random_uniform([10, 3], -1., 1.))

# 편향을 각각 각 레이어의 아웃풋 갯수로 설정합니다.
# b1 은 히든 레이어의 뉴런 갯수로, b2 는 최종 결과값 즉, 분류 갯수인 3으로 설정합니다.
b1 = tf.Variable(tf.zeros([10]))
b2 = tf.Variable(tf.zeros([3]))

# 신경망의 히든 레이어에 가중치 W1과 편향 b1을 적용합니다
L1 = tf.add(tf.matmul(X, W1), b1)
L1 = tf.nn.relu(L1)

# 최종적인 아웃풋을 계산합니다.
# 히든레이어에 두번째 가중치 W2와 편향 b2를 적용하여 3개의 출력값을 만들어냅니다.
model = tf.add(tf.matmul(L1, W2), b2, name='model')


prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

saver = tf.train.import_meta_graph("./model/dnn_final.ckpt.meta")
saver.restore(sess, './model/dnn_final.ckpt')
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))


Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./model/dnn_final.ckpt
예측값: [0 2 2 0 0 2]
실제값: [0 1 2 0 0 2]


### Keras 모델 저장

In [8]:
import keras
import keras.models as models
import keras.layers as layers
import numpy as np

x_data = np.array(
    [[0, 0], [1, 0], [1, 1], [0, 0], [0, 0], [0, 1]])

y_data = np.array([
    [1, 0, 0],  # 기타
    [0, 1, 0],  # 포유류
    [0, 0, 1],  # 조류
    [1, 0, 0],
    [1, 0, 0],
    [0, 0, 1]
])

model = models.Sequential()
model.add(layers.Dense(3, activation='softmax', input_shape=(2,), name="layer-1"))
model.compile(optimizer='SGD', loss='mse', metrics=['accuracy'])

callbaks=[
    keras.callbacks.TensorBoard(
        log_dir = './log2',
    )
]

model.fit(x_data, y_data, epochs=200, verbose=0, callbacks=callbaks)



pred = np.argmax(model.predict(x_data), axis=1)
target = np.argmax(y_data, axis=1)
print('예측값:', pred)
print('실제값:', target)
print('정확도: %.2f'% (np.sum(pred == target)/target.shape[0]*100))


예측값: [0 0 1 0 0 1]
실제값: [0 1 2 0 0 2]
정확도: 50.00


In [10]:
model.save('./model2/model.h5')

### Keras Model 읽기


In [11]:
from keras.models import load_model
model = load_model('./model2/model.h5')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer-1 (Dense)              (None, 3)                 9         
Total params: 9
Trainable params: 9
Non-trainable params: 0
_________________________________________________________________


In [12]:
x_data = np.array(
    [[0, 0], [1, 0], [1, 1], [0, 0], [0, 0], [0, 1]])


pred = np.argmax(model.predict(x_data), axis=1)
target = np.argmax(y_data, axis=1)
print('예측값:', pred)
print('실제값:', target)
print('정확도: %.2f'% (np.sum(pred == target)/target.shape[0]*100))

예측값: [0 0 1 0 0 1]
실제값: [0 1 2 0 0 2]
정확도: 50.00
